In [1]:
# 合并数据集
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
# 数据库风格的DataFrame合并

In [3]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [4]:
# inner join，只要key(因为没有指定，所以就按重叠的列名来)相等的地方就排列组合，所以新的df里key等于b有3个值。
# 默认等于pd.merge(df1, df2, on='key'),on后面可以跟数组，但必须是2个DataFrame同时拥有。
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # 指定merge依据的key
# 因为c和d不在lkey中，无法和rkey匹配，所以合并结果没有。

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [6]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer') # 不匹配的地方自动填空值

,data1,lkey,data2,rkey
0,0.0,b,1.0,b
1,1.0,b,1.0,b
2,6.0,b,1.0,b
3,2.0,a,0.0,a
4,4.0,a,0.0,a
5,5.0,a,0.0,a
6,3.0,c,NaN,NaN
7,NaN,NaN,2.0,d


In [7]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
pd.merge(df1, df2, on='key', how='left') # left join，c会出现，data2自动填充NA。

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [8]:
pd.merge(df1, df2, how='inner') # c会被剔除

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [9]:
# 根据多个键做合并
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer') # outer join，配不上的自动填充NA。

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [10]:
pd.merge(left, right, on='key1') # key2重名如何处理？自动补_x和_y。

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [11]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) # 手工指定后缀

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [12]:
# merge函数的参数
# left：       参与合并的左侧DataFrame
# right：      参与合并的右侧DataFrame
# how：        inner/outer/left/right，默认inner。
# on：         用于连接的列名。必须存在于左右两个DataFrame对象中。如果未指定，且其他连接键也未指定，
#              则以left和right列名的交集作为连接键。
# left_on：    左侧DataFrame中用作连接键的列
# right_on：   右侧DataFrame中用作连接键的列
# left_index： 将左侧的行索引用作其连接键
# right_index：将右侧的行索引用作其连接键
# sort：       根据连接键对合并后的数据进行排序，默认为True。
#              在处理大数据集时，禁用该选项可获得更好的性能。
# suffixes：   字符串值元组。用于追加到重叠列名的末尾。默认是('_x', '_y')。

In [13]:
# 索引上的合并

In [14]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                   'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [15]:
pd.merge(left1, right1, left_on='key', right_index=True) # 使用左边的key列和右边的索引做合并

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [16]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # 使用outer join

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [17]:
left = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                  'key2': [2000, 2001, 2002, 2001, 2002],
                 'data': np.arange(5.)})
right = DataFrame(np.arange(12).reshape((6, 2)),
                  index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                         [2001, 2000, 2000, 2000, 2001, 2002]],
                  columns=['event1', 'event2'])
print(left)
print(right)

   data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [18]:
 pd.merge(left, right, left_on=['key1', 'key2'], right_index=True) #  因为righth是多重索引，所以lefth要指定多个列。

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [19]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                  index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'],
                   columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [20]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True) # 使用左右索引合并

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [21]:
left2.join(right2, how='outer') # 参考SQL的join，默认使用行索引进行连接。

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [22]:
left1.join(right1, on='key') # 指定join的基准列

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [23]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'],
                    columns=['New York', 'Oregon'])
left2.join([right2, another]) # 沿着列合并

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [24]:
left2.join([right2, another], how='outer') # 多个DataFrame根据index做outer join

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [25]:
# 轴向连接

In [26]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [27]:
np.concatenate([arr, arr], axis=1) # 沿着列方向合并2个数组

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [28]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g']) 
pd.concat([s1, s2, s3]) # 一维数组线性相加，这里默认axis=0

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [29]:
pd.concat([s1, s2, s3], axis=1) # 因为axis=1，所以变成沿着列方向合并，变成3列，自动填充NA。

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [30]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [31]:
pd.concat([s1, s4], axis=1, join='inner') # 两个Series沿着列合并变成2列

,0,1
a,0,0
b,1,5


In [32]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) # 指定行索引，没有的自动填充NA

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [33]:
# 使用keys构造层次化索引，one/two/three分别对应s1/s1/s3构造的行。
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [34]:
result.unstack() # 索引转移为列

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [35]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) # 沿着列的方向合并，keys变成columns。

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [36]:
df1 = DataFrame(np.arange(6).reshape(3, 2),
                index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2),
                index=['a', 'c'],
                columns=['three', 'four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2']) # level1/2分别对应df1/2的列

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [37]:
pd.concat({'level1': df1, 'level2': df2}, axis=1) # 字典的key对应多重索引的列名

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [38]:
pd.concat([df1, df2], 
          axis=1,
          keys=['level1', 'level2'],
          names=['upper', 'lower']) # 列索引的名字

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [39]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print(df2)

          a         b         c         d
0  0.636348  0.530036 -2.009787  0.046901
1 -0.106937 -1.725317 -0.567457 -1.297090
2 -0.699975 -0.951912 -2.016601 -0.013257
          b         d         a
0  0.244743 -0.045252 -0.458518
1  1.047011 -1.296364 -0.530735


In [40]:
pd.concat([df1, df2], ignore_index=True) # 忽略索引的话，按行叠加顺序重新计算索引，不然会复用原来的索引。

,a,b,c,d
0,0.636348,0.530036,-2.009787,0.046901
1,-0.106937,-1.725317,-0.567457,-1.297090
2,-0.699975,-0.951912,-2.016601,-0.013257
3,-0.458518,0.244743,NaN,-0.045252
4,-0.530735,1.047011,NaN,-1.296364


In [41]:
# concat函数的参数
# objs:             参与连接的pandas对象的列表或字典。唯一必须的参数。
# axis：            指明连接的轴向，默认为0。
# join：            inner或outer，也就是取交集还是并集。
# join_axes：       指明用于其它n-1条轴的索引，不执行并集/交集运算。
# keys：            与连接对象有关的轴，用于形成连接轴向上的层次化索引。
#                   可以是任意值的列表或数组、元组数组、数组列表。
# levels：          指定用作层次化索引各级别上的索引，如果设置了keys。
# names：           用于创建分层级别的名称
# verify_integrity：检查结果对象新轴上的重复情况。
#                   如果发现则引发异常。默认（False）允许重复。

In [42]:
# 合并重叠数据

In [43]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print(b)
np.where(pd.isnull(a), b, a) # 如果a对应索引为空就取b的值

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [44]:
# combine_first自动对齐并根据是否NAN选择用哪个值。
# b的fedc与a的dcba混合，ba使用a的值，fe使用b的值，dc如果b为NAN就用a的dc。
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [45]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df1.combine_first(df2) # 按行和列进行合并

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
